In [6]:
import pandas as pd

# Example: load 2023 season play-by-play

def show_columns(filename): 
    df = pd.read_csv(filename)
    
    # https://github.com/nflverse/nflverse-data/releases/tag/pbp
    
    print(df.columns)
    df.head()
    
    for column_name in df:
        print(f"{column_name}")


In [19]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values

def load_data(filename): 
    
    DB_CONFIG = {
        "dbname": "nfl_historical",
        "user": "postgres",
        "password": "postgres",
        "host": "localhost",
        "port": 5432,
    }
    
    # === READ CSV ===
    df = pd.read_csv(filename)
    
    # === CONNECT TO DATABASE ===
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    
    # === CREATE TABLES IF NOT EXIST ===
    cur.execute("""
    CREATE TABLE IF NOT EXISTS games (
        id TEXT PRIMARY KEY,
        home_team TEXT NOT NULL,
        away_team TEXT NOT NULL,
        description TEXT NOT NULL,
        game_date date NOT NULL,
        season_type TEXT NOT NULL,
        week smallint NOT NULL
    );
    """)
    
    cur.execute("""
    CREATE TABLE IF NOT EXISTS plays (
        id TEXT PRIMARY KEY,
        game_id TEXT NOT NULL REFERENCES games(id) ON DELETE CASCADE,
        posteam TEXT NOT NULL,
        posteam_type TEXT NOT NULL,
        defteam TEXT NOT NULL,
        total_home_score int,
        total_away_score int,
        qtr smallint,
        quarter_seconds_remaining float,
        game_seconds_remaining float
    );
    """)
    
    conn.commit()
    
    # === INSERT INTO games ===
    # Use UPSERT to avoid duplicates (in case same game appears multiple times)
    games = df[["game_id", "home_team", "away_team", "desc", "game_date", "season_type", "week", ]].drop_duplicates(subset=["game_id"])
    
    execute_values(
        cur,
        """
        INSERT INTO games (id, home_team, away_team, description, game_date, season_type, week)
        VALUES %s
        ON CONFLICT (id) DO UPDATE SET 
          home_team = EXCLUDED.home_team,
          away_team = EXCLUDED.away_team,
          game_date = EXCLUDED.game_date,
          season_type = EXCLUDED.season_type,
          week = EXCLUDED.week
        """,
        list(games.itertuples(index=False, name=None))
    )
    
    # === INSERT INTO plays ===
    plays = df[["play_id", "game_id", "posteam", "posteam_type", "defteam", "total_home_score", "total_away_score", "qtr", "quarter_seconds_remaining", "game_seconds_remaining"]].drop_duplicates(subset=["play_id"])
    
    plays["play_id"] = plays["game_id"].astype(str) + "_" + plays["play_id"].astype(str)
    
    execute_values(
        cur,
        """
        INSERT INTO plays (
            id, 
            game_id, 
            posteam, 
            posteam_type, 
            defteam,
            total_home_score, 
            total_away_score, 
            qtr,
            quarter_seconds_remaining,
            game_seconds_remaining
        )
        VALUES %s
        ON CONFLICT (id) DO UPDATE SET
          game_id = EXCLUDED.game_id,
          posteam = EXCLUDED.posteam,
          posteam_type = EXCLUDED.posteam_type,
          defteam = EXCLUDED.defteam,
          total_home_score = EXCLUDED.total_home_score,
          total_away_score = EXCLUDED.total_away_score,
          qtr = EXCLUDED.qtr,
          quarter_seconds_remaining = EXCLUDED.game_seconds_remaining,
          game_seconds_remaining = EXCLUDED.game_seconds_remaining
        """,
        list(plays.itertuples(index=False, name=None))
    )
    
    # === COMMIT AND CLEANUP ===
    conn.commit()
    cur.close()
    conn.close()
    
    print("Data import completed successfully.")
    print(filename)


In [20]:

#show_columns("./play_by_play_2006.csv");

In [24]:
#load_data("./play_by_play_2006.csv")
#load_data("./play_by_play_2020.csv")
#load_data("./play_by_play_2021.csv")
load_data("./play_by_play_2022.csv")
#load_data("./play_by_play_2023.csv")
#load_data("./play_by_play_2024.csv")

FileNotFoundError: [Errno 2] No such file or directory: './play_by_play_2022.csv'